In [1]:
import numpy as np
import pyvista as pv
from mpi4py import MPI
from dolfinx import mesh, plot, fem, io
from dolfinx.fem.petsc import LinearProblem
import ufl
from ufl import TrialFunction, TestFunction, dx, grad, inner
from petsc4py import PETSc
import scipy.sparse

# Paramètres
L1 = 1.0  # Longueur de l'intervalle
nx = 50   # Nombre de subdivisions

# Création du maillage 1D
domain = mesh.create_interval(MPI.COMM_WORLD, nx, [0, L1])

# Tracé du maillage en utilisant PyVista
plotter = pv.Plotter()
topology, cell_types, geometry = plot.vtk_mesh(domain)
grid = pv.UnstructuredGrid(topology, cell_types, geometry)
plotter.add_mesh(grid, show_edges=True)
plotter.add_text("Maillage du domaine", font_size=12, color="grey", position="upper_edge")  # Ajout d'un texte ("Maillage du domaine") en gris en haut du plot
plotter.show_bounds(color="grey")  # Affichage des limites du plot avec des lignes grises
plotter.add_axes(color="grey")  # Ajout des axes du plot avec des lignes grises
plotter.set_background("black")  # Définition de l'arrière-plan du plot en noir
plotter.show()

Widget(value='<iframe src="http://localhost:62298/index.html?ui=P_0x169acf9e0_0&reconnect=auto" class="pyvista…

In [3]:
plotter.view_zx() # Vue XY (vue par défaut)

In [4]:
# Définir l'espace fonctionnel
V = fem.functionspace(domain, ("CG", 1))

# Définir les fonctions de test et d'essai
thetat = ufl.TestFunction(V)
delta_theta = ufl.TrialFunction(V)

# Définir les formes bilinéaires
a_k = ufl.dot(ufl.grad(delta_theta), ufl.grad(thetat)) * ufl.dx
a_g = delta_theta * thetat * ufl.dx

# Assembler les matrices
K = fem.petsc.assemble_matrix(fem.form(a_k))
K.assemble()

G = fem.petsc.assemble_matrix(fem.form(a_g))
G.assemble()

In [5]:
# Conversion des matrices PETSc en matrices SciPy
def petsc_to_scipy(petsc_mat):
    AI, AJ, AV = petsc_mat.getValuesCSR()
    return scipy.sparse.csr_matrix((AV, AJ, AI), shape=petsc_mat.getSize())

K_scipy = petsc_to_scipy(K)
G_scipy = petsc_to_scipy(G)

# Conversion en matrices denses pour affichage (pour des petites matrices)
K_dense = K_scipy.toarray()
G_dense = G_scipy.toarray()

# Affichage des matrices denses
print("Matrice de rigidité K (dense):")
print(K_dense)

print("Matrice de précontrainte G (dense):")
print(G_dense)

Matrice de rigidité K (dense):
[[ 50. -50.   0. ...   0.   0.   0.]
 [-50. 100. -50. ...   0.   0.   0.]
 [  0. -50. 100. ...   0.   0.   0.]
 ...
 [  0.   0.   0. ... 100. -50.   0.]
 [  0.   0.   0. ... -50. 100. -50.]
 [  0.   0.   0. ...   0. -50.  50.]]
Matrice de précontrainte G (dense):
[[0.00666667 0.00333333 0.         ... 0.         0.         0.        ]
 [0.00333333 0.01333333 0.00333333 ... 0.         0.         0.        ]
 [0.         0.00333333 0.01333333 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.01333333 0.00333333 0.        ]
 [0.         0.         0.         ... 0.00333333 0.01333333 0.00333333]
 [0.         0.         0.         ... 0.         0.00333333 0.00666667]]


In [6]:
from slepc4py import SLEPc
# Définir le solveur de valeurs propres
eigensolver = SLEPc.EPS().create()
eigensolver.setOperators(K, G)
eigensolver.setProblemType(SLEPc.EPS.ProblemType.GNHEP)

# Définir les paramètres du solveur
eigensolver.setType(SLEPc.EPS.Type.KRYLOVSCHUR)
eigensolver.setWhichEigenpairs(SLEPc.EPS.Which.TARGET_REAL)
eigensolver.setTarget(1.0)

ST = eigensolver.getST()
ST.setType(SLEPc.ST.Type.SINVERT)

# Définir les options PETSc
opts = PETSc.Options()
opts['eps_monitor'] = 1

# Résoudre le problème aux valeurs propres
eigensolver.solve()

# Obtenir les résultats
nconv = eigensolver.getConverged()
print(f"Number of converged eigenpairs: {nconv}")

# Extraction et affichage des valeurs propres réelles
for i in range(nconv):
    # Obtenir la valeur propre
    eigenvalue = eigensolver.getEigenvalue(i)
    print(f"Eigenvalue {i}: {eigenvalue.real}")

Number of converged eigenpairs: 5
Eigenvalue 0: 2.544631172440859e-13
Eigenvalue 1: 9.872851797903968
Eigenvalue 2: 39.530396421925666
Eigenvalue 3: 89.08975451803028
Eigenvalue 4: 158.74663349382962


In [7]:
# Obtenir le nombre d'itérations
iterations = eigensolver.getIterationNumber()
print('Number of iterations:', iterations)

Number of iterations: 1


In [8]:
# Amplification des modes et visualisation
scale_factor = 1  # Facteur de mise à l'échelle pour la visualisation

# Préparer le maillage pour PyVista
topology, cell_types, geometry = plot.vtk_mesh(domain)

In [9]:
# Création du fichier XDMF
xdmf_file = io.XDMFFile(MPI.COMM_WORLD, "modes.xdmf", "w")
xdmf_file.write_mesh(domain)

In [10]:
# Préparer le mode_theta pour la visualisation et l'enregistrement
mode_theta = fem.Function(V)

for i in range(0, nconv):  # Ignorer le premier mode si nécessaire
    lambda_r = eigensolver.getEigenvalue(i)
    eigensolver.getEigenvector(i, mode_theta.vector)
    
    # Convertir PETSc Vec en tableau NumPy
    mode_values = mode_theta.vector.array
    
    # Amplifier les valeurs pour la visualisation
    amplified_values = mode_values * scale_factor
    
    # Ajouter les données de déformation au maillage
    grid = pv.UnstructuredGrid(*plot.vtk_mesh(domain))
    grid.point_data[f"Mode_{i}"] = amplified_values
    
    # Créer un objet PyVista Warp pour visualiser les déformations
    warped = grid.warp_by_scalar(f"Mode_{i}", factor=scale_factor)  # Ajustez le facteur de mise en forme si nécessaire
    
    # Tracé des modes déformés
    plotter = pv.Plotter()
    plotter.add_mesh(warped, show_edges=True, cmap="turbo", show_scalar_bar=False) 
    plotter.add_text(f"Mode {i}", font_size=12, color="black", position="upper_edge")      
    scalar_bar = plotter.add_scalar_bar(
        color="black",  # Couleur du texte de la barre scalaire
        title="u",  # Titre de la barre scalaire
        title_font_size=24,  # Taille de police du titre
        label_font_size=22,  # Taille de police des étiquettes
        shadow=False,  # Désactiver l'ombre
        n_labels=5,  # Nombre d'étiquettes sur la barre scalaire
        italic=False,  # Texte en italique
        font_family="arial",  # Famille de police du texte
        vertical=True  # Orientation verticale de la barre scalaire
    )
    plotter.show_bounds(color="black")
    plotter.add_axes(color="black")
    plotter.set_background("grey")
    plotter.show()
    
    # Écriture du mode amplifié dans le fichier XDMF
    mode_function = fem.Function(V)
    mode_function.vector.setArray(amplified_values)
    xdmf_file.write_function(mode_function, t=float(i))

xdmf_file.close()

print("Modes written to XDMF file.")

Widget(value='<iframe src="http://localhost:62298/index.html?ui=P_0x286aac830_1&reconnect=auto" class="pyvista…

Widget(value='<iframe src="http://localhost:62298/index.html?ui=P_0x28f7533e0_2&reconnect=auto" class="pyvista…

Widget(value='<iframe src="http://localhost:62298/index.html?ui=P_0x28f786600_3&reconnect=auto" class="pyvista…

Widget(value='<iframe src="http://localhost:62298/index.html?ui=P_0x28f786270_4&reconnect=auto" class="pyvista…

Widget(value='<iframe src="http://localhost:62298/index.html?ui=P_0x2a9d0a330_5&reconnect=auto" class="pyvista…

Modes written to XDMF file.
